# 跳跃收益因子

## 导入模块

In [1]:
import numpy as np
import pandas as pd
import feather

## 读入数据

In [2]:
jump = feather.read_dataframe('../data/jump/jump.feather')

### 交易日历

In [3]:
trade_date = jump['date'].sort_values().unique()
mes = pd.date_range(start=trade_date[0], end=trade_date[-1]+pd.DateOffset(months=1), freq='1ME')
adj_date = np.array([], dtype=np.datetime64)
calc_start_date = np.array([], dtype=np.datetime64)

for me in mes:
    trade_date_before = trade_date[trade_date <= me]
    ad = trade_date_before[-1]
    adj_date = np.append(adj_date, ad)
    if (len(trade_date_before) > 20):
        csd = trade_date_before[-20]
    else:
        csd = trade_date_before[0]
    calc_start_date = np.append(calc_start_date, csd)

adj_date[-1] = pd.Timestamp('2024-06-28')

## 跳跃收益因子

In [4]:
ret_jump = None
for ad, csd in zip(adj_date, calc_start_date):
    jump_date = jump[(jump['date'] <= ad) & (jump['date'] >= csd)]
    ret_jump_date = jump_date.groupby('issue')['ret_jump'].sum().reset_index()
    ret_jump_date['date'] = ad
    ret_jump = pd.concat([ret_jump, ret_jump_date])
feather.write_dataframe(ret_jump, '../data/ret_jump.feather')